In [1]:
!python -V

Python 3.9.18


In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import pandas as pd

In [3]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error

In [4]:
import mlflow

mlflow.set_tracking_uri("sqlite:///../mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/home/pytholic/mlops-zoomcamp-pytholic/02-experiment-tracking/mlruns/1', creation_time=1702972736474, experiment_id='1', last_update_time=1702972736474, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [5]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    # Calculate duration
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    # Filter between 1 min and 60 mins
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']    
    df[categorical] = df[categorical].astype(str)
    
    return df

In [6]:
df_train = read_dataframe("../data/green_tripdata_2021-01.parquet")
df_val = read_dataframe("../data/green_tripdata_2021-02.parquet")

In [7]:
len(df_train), len(df_val)

(73908, 61921)

**Note:** There was an issue in applying separate transforms i.e. X_train and X_val yielded different shapes. So I applied transform after combining them and then separated them again. 

In [8]:
df_combined = pd.concat([df_train, df_val], axis=0)

In [9]:
df_combined['PU_DO'] = df_combined['PULocationID'] + '_' + df_combined['DOLocationID']

In [10]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()
combined_dicts = df_combined[categorical + numerical].to_dict(orient='records')
X_combined = dv.fit_transform(combined_dicts)

In [11]:
X_train = X_combined[:73908, :]
X_val = X_combined[73908:, :]

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.758715206100507

In [14]:
with open("../models/lin_reg.bin", "wb") as f_out:
    pickle.dump((dv, lr), f_out)

Save the model.

In [15]:
# Method 1
with mlflow.start_run():

    mlflow.set_tag("developer", "pytholic")

    mlflow.log_param("train-data-path", "../data/green_tripdata_2021-01.parquet")
    mlflow.log_param("val-data-path", "../data/green_tripdata_2021-02.parquet")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="../models/lin_reg.bin", artifact_path="models_pickle")

In [16]:
import xgboost as xgb

In [17]:
# Prepare data for xgboost
train = xgb.DMatrix(X_train, label=y_train)
val = xgb.DMatrix(X_val, label=y_val)

In [18]:
# Custom callback class to log metrics during training
class LogMetricsCallback(xgb.callback.TrainingCallback):
    def __init__(self, eval_set, y_val):
        self.eval_set = eval_set
        self.y_val = y_val

    def after_iteration(self, model, epoch, evals_log):
        if epoch % 10 == 0:  # Log every 10 epochs, adjust as needed
            y_pred = model.predict(self.eval_set)
            rmse = mean_squared_error(y_val, y_pred, squared=False)
            mlflow.log_metric("rmse", rmse, step=epoch)

# Method 2
with mlflow.start_run():

    # Use best params to train model
    params = {
        "learning_rate": 0.20472169880371677,
        "max_depth": 17,
        "min_child_weight": 1.2402611720043835,
        "objective": "reg:linear",
        "reg_alpha": 0.28567896734700793,
        "reg_lambda": 0.004264404814393109,
        "seed": 42
    }

    mlflow.log_params(params)

    booster = xgb.train(
                params=params,
                dtrain=train,
                num_boost_round=1000,
                evals=[(val, "validation")],
                early_stopping_rounds=50,
                callbacks=[LogMetricsCallback(eval_set=val, y_val=y_val)]
            )
    
    y_pred = booster.predict(val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("../models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("../models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

Make predictions.

In [19]:
logged_model = 'runs:/82f2550f1f434fb19c7bb469e8cbd65b/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [20]:
loaded_model

In [21]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

In [22]:
xgboost_model

In [23]:
y_pred = xgboost_model.predict(val)
y_pred[:10]

Train different models.

In [24]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (LinearRegression, Lasso, Ridge, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "../data/green_tripdata_2021-01.parquet")
        mlflow.log_param("val-data-path", "../data/green_tripdata_2021-02.parquet")
        mlflow.log_artifact("../models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

2023/12/20 05:04:47 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2023/12/20 05:04:49 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/pytholic/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
2023/12/20 05:04:55 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2023/12/20 05:05:05 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2023/12/20 05:05:12 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
/home/pytholic/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/sklearn/svm/_c